In [71]:
import pdfplumber
import pandas as pd
import numpy as np
import os
import textEx
from datetime import datetime

In [72]:
def readAllHongTaiPDF(dir:str) -> pd.DataFrame:
    column_names = ['规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位','规格', '貨名', '客户价', '仓位' ]
    all_pages_df = pd.DataFrame(columns=column_names)
    all_pages_df.insert(4, 'date',None) 
    all_pages_df.insert(9, 'date2',None)  
    all_pages_df.insert(14, 'date3', None)  
    all_pages_df.insert(19, 'date4', None)  
    all_pages_df.insert(24, 'date5', None)  

    fail_df=[]
    for filename in os.listdir(dir):
        if filename.endswith('.pdf'):
            file_path = os.path.join(dir, filename)
            with pdfplumber.open(file_path) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        try:
                            df = pd.DataFrame(table[1:], columns=column_names)
                            date=filename.replace("詳細報價單","").replace(".pdf","")
                            df.insert(4, 'date', date)  
                            df.insert(9, 'date2', date)  
                            df.insert(14, 'date3', date)  
                            df.insert(19, 'date4', date) 
                            df.insert(24, 'date5', date)  
                        except:
                            df=pd.DataFrame(table[1:])
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
                            fail_df.append(filename)
    df1 = all_pages_df.iloc[:, :5]
    df2 = all_pages_df.iloc[:, 5:10]
    df3 = all_pages_df.iloc[:, 10:15]
    df4 = all_pages_df.iloc[:, 15:20]
    df5 = all_pages_df.iloc[:, 20:25]
    df2.columns = df1.columns
    df3.columns = df1.columns
    df4.columns = df1.columns
    df5.columns = df1.columns
    df_concatenated_corrected = pd.concat([df1, df2, df3,df4,df5], axis=0).reset_index(drop=True)
    df_concatenated_corrected=df_concatenated_corrected.dropna()
    df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
    df_cleaned = df.dropna(subset=["貨名"], how='any').reset_index(drop=True)
    df_cleaned = df_cleaned[df_cleaned["仓位"] != "倉"].reset_index(drop=True)
    df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    column_names_new = ['规格', '貨名牌子', '客户价','仓位']
    all_pages_df = pd.DataFrame(columns=column_names_new)
    all_pages_df.insert(4, 'date',None) 
    for filename in set(fail_df):
        filepath = os.path.join(dir, filename)
        with pdfplumber.open(filepath) as pdf:
                for page in pdf.pages:
                    table = page.extract_table()
                    if table:
                        try:
                            df = pd.DataFrame(table[1:], columns=column_names_new)
                            date=filename.replace("詳細報價單(新)","").replace(".pdf","")
                            df.insert(4, 'date', date)
                        except:
                            df=pd.DataFrame(table[1:])
                        try:
                            all_pages_df = pd.concat([all_pages_df, df], ignore_index=True)
                        except:
                            print(f"Error merging data from {filename},{page}")
                            fail_df.append(filename)
    new_column_names = ['规格', '貨名', '客户价', '仓位','date']
    df = all_pages_df.iloc[:, 0:5].rename(columns=dict(zip(all_pages_df.iloc[:, 0:5].columns, new_column_names)))


    dfall = pd.concat([df_cleaned, df], axis=0).reset_index(drop=True)

    return dfall

In [73]:
dfall=readAllHongTaiPDF(r"C:\Users\Balle\Downloads\tradeasy-main\tradeasy-main\quotation\鸿泰")

Error merging data from 21.11.2022.pdf,<Page:1>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:1>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:2>
Error merging data from 詳細報價單(新)-27102022_221027_130509.pdf,<Page:3>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:1>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:2>
Error merging data from 詳細報價單(新)08.12.2022.pdf,<Page:3>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:1>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:2>
Error merging data from 詳細報價單(新)11022023_230211_111325.pdf,<Page:3>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:1>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:2>
Error merging data from 詳細報價單(新)1212_231212_124233.pdf,<Page:3>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:1>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:2>
Error merging data from 詳細報價單(新)13062023.pdf,<Page:3>
Error merging data 

C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\3390486293.py:44: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df_concatenated_corrected.applymap(lambda x: x.strip() if isinstance(x, str) else x).replace('', np.nan)
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\3390486293.py:47: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_cleaned = df_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)


In [74]:
dfall = dfall[dfall["仓位"] != ""].reset_index(drop=True)
dfall = dfall[dfall["仓位"] != "倉位"].reset_index(drop=True)
dfall = dfall[dfall["规格"] != "規格"].reset_index(drop=True)
dfall = dfall[dfall["规格"] != ""].reset_index(drop=True)
dfall = dfall[dfall["规格"].notna()].reset_index(drop=True)

In [122]:
def _specToList(spec :list) -> list:
    spec1 = spec2 = spec3 = spec4 = spec5 = spec6 = ""
    if spec is not None:
        spec = [item for item in spec if item is not None]
        spec.sort()
        if len(spec) > 0:
            spec1 = spec[0]
        if len(spec) > 1:
            spec2 = spec[1]
        if len(spec) > 2:
            spec3 = spec[2]
        if len(spec) > 3:
            spec4 = spec[3]
        if len(spec) > 4:
            spec5 = spec[4]
        if len(spec) > 5:
            spec6 = spec[5]
    else:
        spec1=spec2=spec3=spec4=spec5=spec6=""

    return [spec1,spec2,spec3,spec4,spec5,spec6]
        
import re
from datetime import datetime

def extract_date(filename):
    if isinstance(filename,str):
        # 尝试匹配 YYYYMMDD_hhmmss 格式
        match = re.search(r'\d+_(\d{2})(\d{2})(\d{2})_\d+', filename)
        if match:
            year, month, day = match.groups()
            year = '20' + year  # 假设年份为 20xx 年
            date_str = f"{year}-{month}-{day}"
            return datetime.strptime(date_str, '%Y-%m-%d').date()
        # 匹配 DD.MM.YYYY
        match = re.search(r'(\d{2})\.(\d{1,2})\.(\d{4})', filename)
        if match:
            day, month, year = match.groups()
            date_str = f"{year}-{month.zfill(2)}-{day}"
            return datetime.strptime(date_str, '%Y-%m-%d').date()

        # 尝试匹配 DDMMYYYY 格式
        match = re.search(r'(\d{2})(\d{2})(\d{4})', filename)
        if match:
            day, month, year = match.groups()
            date_str = f"{year}-{month}-{day}"
            return datetime.strptime(date_str, '%Y-%m-%d').date()
    else:
        return filename

    return filename

def getHongTaiQuote(df: pd.DataFrame) -> pd.DataFrame:
    df['specs'] = df['貨名'].apply(textEx.getSpec)  
    df['brand'] = df['貨名'].apply(textEx.getBrand)
    for index, row in df.iterrows():
        if row['brand'] is not None:
            df.at[index, 'productName'] = row['貨名'].replace(row['brand'], '')
        else:
            df.at[index, 'productName'] = row['貨名']
    df['productTag'] = df['貨名'].apply(textEx.getProduct)
    df['supplier'] = '鸿泰'
    df['category'] = df['貨名'].apply(textEx.getCategory)
    df['packing']=df['规格']
    df['origin'] = df['貨名'].apply(textEx.getCountry)
    df['effectiveDate'] = df['date'].apply(extract_date)
    df[['spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6']] = df['specs'].apply(lambda x: pd.Series(_specToList(x)))
    df['客户价'] = df['客户价'].astype(str)
    df['price'] = df['客户价'].str.replace('$','').replace(" ","")
    df['weightUnit'] = df['客户价'].apply(textEx.getWeightUnit)
    df['仓位'] = df['仓位'].astype(str)
    df['warehouse']=df['仓位'].apply(textEx.getWarehoue)
    df['notes']= None

    df = df.explode('warehouse')
    df['warehouse'] = df['warehouse'].astype(str)

    required_columns = [
    'productName', 'productTag', 'supplier', 'category', 'packing',
    'origin', 'brand', 'effectiveDate', 'spec1', 'spec2', 'spec3',
    'spec4', 'spec5', 'spec6', 'price', 'weightUnit', 'warehouse', 'notes'
    ]
    df=df[required_columns]
    return df

In [123]:
dfout=getHongTaiQuote(dfall)

C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataF

[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['93廠']
[]
no _match is found
[]
no _match is found
['240/260']
['260/280']
['+300g', '280/300']
['+300g', '300/320']
[]
no _match is found
['去皮']
['帶皮']
['去皮']
['+300g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['240/260']
['260/280']
['+300g', '280/300']
['+300g', '300/320']
[]
no _match is found
['去皮']
['帶皮']
['去皮']
['+300g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no 

C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataF

[]
no _match is found
['93廠']
[]
no _match is found
[]
no _match is found
['240/260']
['260/280']
['+300g', '280/300']
['+300g', '300/320']
[]
no _match is found
['去皮']
['帶皮']
['去皮']
['+300g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['3/5']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['直']
[]
no _match is found
[]
no _match is found
[]
no _match is found
['3/5', '93廠']
[]
no _match is found
[]
no _match is found
['240/260']
['260/280']
['+300g', '280/300']
['+300g', '300/320']
[]
no _match is found
['去皮']
['帶皮']
['去皮']
['+300g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['+900g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['93廠']
[]


C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataF

...

[]
no _match is found
[]
no _match is found
[]
no _match is found
['直', '+900g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['245l廠']
['278廠']
['245j廠']
['245c廠']
[]
no _match is found
['245c廠']
['245j廠']
['245l廠']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['11/15']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
n

C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataF

[]
no _match is found
['+900g']
[]
no _match is found
[]
no _match is found
['+200g', '220/240']
['+200g', '220/240']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['+200g', '220/240']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['3條庄']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['3/4']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['直', '+900g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['3支骨']
[]
no _match is found
[]
no _match is found
[]
no _match is found
['278廠']
['245j廠']
['245c廠']
[]
no _match is found
['278廠']
['245c廠']
[]
no _match is found
[]
no _match is found
[]
n

C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  result_dict = df_new.to_dict('list')
C:\Users\Balle\AppData\Local\Temp\ipykernel_36256\532258124.py:62: UserWarning: DataF

[]
no _match is found
[]
no _match is found
[]
no _match is found
['+200g']
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
[]
no _match is found
['2.5k+']
[]
no _match is found
[]
no _match is found
[]
no _match is found
['240/260']
['260/280']
['+300g', '280/300']
['+300g', '300/320']
[]
no _match is found
['去皮']
['帶皮']
['去皮']
[]
no _match is found
['+300g']
[]
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match is found
no _match i

In [126]:
dfout.to_csv("鸿泰.csv")